In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from backtest import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pprint
from pprint import PrettyPrinter

In [ ]:
plt.rcParams['figure.figsize'] = [21, 13]
pd.set_option('precision', 10)
pp = PrettyPrinter()

In [ ]:
exchange = 'binance'
user = 'your_user_name'
settings = load_settings(exchange, user)
s = 'ETHUSDT'
n_days = 10

In [ ]:
#load cache if any
#df = pd.read_csv('ETHUSDT_agg_trades_binance_10_days_2021-01-16_price_step_0_2.csv')

In [ ]:
#otherwise
adf = await load_trades(exchange, user, s, n_days)

# prep df for backtesting
n_days = (adf.timestamp.iloc[-1] - adf.timestamp.iloc[0]) / 1000 / 60 / 60 / 24
print('ndays', n_days)
df = prep_df(adf, settings)

In [ ]:
df.price.iloc[::100].plot()

In [ ]:
# plotting method

def plot_tdf_(df_, tdf_, side_: int = 0, liq_thr=0.1):
    df_.loc[tdf_.index[0]:tdf_.index[-1]].price.plot(style='y-')
    if side_ >= 0:
        longs = tdf_[tdf_.side == 'long']
        le = longs[longs.type == 'entry']
        lc = longs[longs.type == 'close']
        ls = longs[longs.type == 'stop_loss']
        ls.price.plot(style='gx')
        le.price.plot(style='b.')
        longs.pos_price.plot(style='b--')
        if 'close_price' in longs.columns:
            longs.close_price.plot(style='r--')
        lc.price.plot(style='ro')
    if side_ <= 0:
        shrts = tdf_[tdf_.side == 'shrt']
        se = shrts[shrts.type == 'entry']
        sc = shrts[shrts.type == 'close']
        ss = shrts[shrts.type == 'stop_loss']
        ss.price.plot(style='gx')
        se.price.plot(style='r.')
        shrts.pos_price.plot(style='r--')
        if 'close_price' in shrts.columns:
            shrts.close_price.plot(style='b--')
        sc.price.plot(style='bo')
    if 'liq_price' in tdf_.columns:
        tdf_.liq_price.where((tdf_.price - tdf_.liq_price).abs() / tdf_.price < liq_thr, np.nan).plot(style='k--')


In [ ]:
# check jackrabbit results if desired
'''
rfilepath = 'backtesting_results/binance/2021-01-16T22:17:15_10/'
'''

In [ ]:
'''
rdf = pd.read_csv(rfilepath + 'results.csv')
rdfs = rdf.sort_values('gain', ascending=False)
rdfs.head(30)
'''

In [ ]:
# load backtesting settings used in jackrabbit and set highest gain one, and load trades
'''
best = sort_dict_keys(dict(rdfs.iloc[0]))
key = best['Unnamed: 0']
settings = json.load(open(rfilepath + 'backtesting_settings.json'))
for k in best:
    if k in settings:
        print(k, best[k])
        settings[k] = best[k]
best

tdf = pd.read_csv(f'{rfilepath}trades/{key}.csv').set_index('trade_id')
'''

In [ ]:
# otherwise load settings and run backtest
'''
settings = json.load(open(f'backtesting_settings/{exchange}/backtesting_settings.json'))
sort_dict_keys(settings)
'''

In [ ]:
# adjust settings if desired
'''
settings['max_markup'] = 0.014
settings['min_markup'] = 0.0002
settings['liq_diff_threshold'] = 0.04
'''

In [ ]:
# run single backtest
'''
start_time = time()
trades = backtest(df, settings)
print(f'seconds elapsed {time() - start_time:.2f}')
tdf = pd.DataFrame(trades).set_index('trade_id')
'''

In [ ]:
# analyze results
longs = tdf[tdf.side == 'long']
shrts = tdf[tdf.side == 'shrt']
le = longs[longs.type == 'entry']
lc = longs[longs.type == 'close']
se = shrts[shrts.type == 'entry']
sc = shrts[shrts.type == 'close']

margin_max = (tdf.pos_size.abs() / tdf.pos_price).max() / settings['leverage']
biggest_pos_size = tdf.pos_size.abs().max()
pnl_sum = tdf.pnl.sum()
stop_loss_closes = tdf[tdf.type == 'stop_loss']
loss_sum = stop_loss_closes.pnl.sum()
gain = (settings['margin_limit'] + pnl_sum) / settings['margin_limit']
closest_liq = ((tdf.price - tdf.liq_price).abs() / tdf.price).min()
n_stop_loss = len(stop_loss_closes)
n_days = settings['n_days']
average_daily_gain = gain ** (1 / n_days)
closes = tdf[tdf.type == 'close']
print('margin_max', margin_max)
print('biggest qty', tdf.qty.abs().max())
print('pnl_sum', pnl_sum)
print('loss sum', loss_sum)
print('gain', gain)
print('n_days', n_days)
print('average_daily_gain', average_daily_gain)
print('n trades', len(tdf))
print('n closes', len(closes))
print('biggest_pos_size', biggest_pos_size)
print('closest liq', closest_liq)
print('n stop loss closes', n_stop_loss)

In [ ]:
plot_tdf_(df, tdf, liq_thr=0.08)

In [ ]:
# visualize behavior
step = 120
i = -step

In [ ]:
i += step
tdfc = tdf.iloc[i:i+step]
plot_tdf_(df, tdf.iloc[i:i+step], liq_thr=0.06)

In [ ]:
tdfcj = tdfc.join(pd.Series(tdfc.price.diff(), name='price_diff'))
tdfcj.head(60)

In [ ]:
tdfcj.tail(60)

In [ ]:
tdf.pnl.cumsum().plot()

In [ ]:
tdf.pos_size.plot()

In [ ]:
closest_liqs = ((tdf.liq_price - tdf.price).abs() / tdf.price).sort_values()
closest_liqs.head()

In [ ]:
i = 0
iloc_ = tdf.index.get_loc(closest_liqs.index[i])
iminus = 400
iplus = 200
tdfc = tdf.iloc[max(0, iloc_-iminus):min(iloc_+iplus, len(tdf) - 1)]
plot_tdf_(df, tdfc)

In [ ]:
tdfc.head(60)

In [ ]:
# inspect biggest trades
qty_abs = tdf.qty.abs().sort_values(ascending=False)
qty_abs.head(10)

In [ ]:
i = 0
iloc_ = tdf.index.get_loc(qty_abs.index[i])
iminus = 400
iplus = 20
tdfc = tdf.iloc[max(0, iloc_-iminus):min(iloc_+iplus, len(tdf) - 1)]
plot_tdf_(df, tdfc)

In [ ]:
ik = -1
stepp = 60

In [ ]:
ik += 1
tdfc.iloc[stepp * ik:stepp * (ik + 1)]

In [ ]:
tdf.tail(30)